In [ ]:
!python3 -V

Python 3.8.15


In [ ]:
!pip3 install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers
!pip install fastai==2.2.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1 MB 1.4 MB/s eta 0:03:59tcmalloc: large alloc 1147494400 bytes == 0x3a7b6000 @  0x7f44e7ef3615 0x5d631c 0x51e4f1 0x51e67b 0x4f7585 0x49ca7c 0x4fdff5 0x49caa1 0x4fdff5 0x49ced5 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x5d7c18 0x5d9412 0x586636 0x5d813c 0x55f3fd 0x55e571 0x5d7cf1 0x49ced5 0x55e571 0x5d7cf1 0x49ec69 0x5d7c18 0x49ca7c 0x4fdff5 0x49ced5
     |█████████████████████████████▏  | 1055.7 MB 1.2 MB/s eta 0:01:28tcmalloc: large alloc 1434370048 bytes == 0x7ee0c000 @  0x7f44e7ef3615 0x5d631c 0x51e4f1 0x51e67b 0x4f7585 0x49ca7c 0x4fdff5 0x49caa1 0x4fdff5 0x49ced5 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x5d7c18 0x5d9412 0x586636 0x5d813c 0x55f3fd 0x55e571 0x5d7cf1 0x49ced5 0x55e571 0x5d7cf1 0x49ec69 0x5d7c18 0x49ca7c 0x4fdff5 0x49ced5
     |█████

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from fastai.text.all import *
import gc

In [ ]:
print(torch. __version__)

1.7.1+cu110


In [ ]:
pretrained_weights = 'gpt2'
tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_weights)
model = GPT2LMHeadModel.from_pretrained(pretrained_weights)

In [ ]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    random.seed(seed_value) # Python
    if use_cuda: 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False
#Remember to use num_workers=0 when creating the DataBunch.

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/YelpDataset/dataframes

AutoDF.pkl		     HealthMedicalDF.pkl  restaurantDF.pkl
beautyandSpaDF.pkl	     HomeServicesDF.pkl   shoppingDF.pkl
EventPlanningServicesDF.pkl  LocalServicesDF.pkl
foodDF.pkl		     NightlifeDF.pkl


In [ ]:
%cp /content/drive/MyDrive/YelpDataset/dataframes/foodDF.pkl /content/

In [ ]:
import pandas as pd
df = pd.read_pickle('foodDF.pkl')

In [ ]:
df = df.rename(columns={'text': 'reviewText'})
df = df.rename(columns={'stars': 'overall'})
df = df.rename(columns={'categories': 'category'})

In [ ]:
df = df[['business_id','name','city','state','overall','category','reviewText']]

In [ ]:
df.head(1)

,business_id,name,city,state,overall,category,reviewText
52819,JX4tUpd09YFchLBuI43lGw,Naked Cyber Cafe & Espresso Bar,Edmonton,AB,4.0,Food,Came here to print off some documents and was having a bit of difficulty. The man working did everything he could to help and we finally got my papers printed off. Great customer service! \n\nI didn't try any refreshments as I had no desire to stay long.


In [ ]:
df['sentenceLength'] = df['reviewText'].str.count(' ') + 1

In [ ]:
df.shape

(299556, 8)

In [ ]:
df.head(2)

,business_id,name,city,state,overall,category,reviewText,sentenceLength
52819,JX4tUpd09YFchLBuI43lGw,Naked Cyber Cafe & Espresso Bar,Edmonton,AB,4.0,Food,Came here to print off some documents and was having a bit of difficulty. The man working did everything he could to help and we finally got my papers printed off. Great customer service! \n\nI didn't try any refreshments as I had no desire to stay long.,47
52820,JX4tUpd09YFchLBuI43lGw,Naked Cyber Cafe & Espresso Bar,Edmonton,AB,4.0,Food,"Naked Cyber & Expresso Bar is a 24 hours cyber cafe. On any given day you will find all kinds of people here, from students doing school work to people that just want to browse the internet. \n\nThey serve coffee, tea and some snacks. Now some people might be mistaken that they can bring their laptops to this place and work away but they do not have wireless internet here. I would assume you can connect your laptop to their internet connection if you asked them but this probably costs quite a bit. They also carry PC's and MACs.\n\nThey also host live music every now and then on their stage...",139


In [ ]:
sample_df = df.sample(frac=0.05, random_state=2021).dropna().reset_index()

In [ ]:
sample_df.shape

(14978, 9)

In [ ]:
del df
gc.collect()

19

In [ ]:
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self.pred[0]

In [ ]:
learn = Learner(dls=None, model=model, loss_func=CrossEntropyLossFlat(), cbs=[DropOutput], metrics=Perplexity()).to_fp16()

### Generated examples of Toys_and_Games_5

In [ ]:
!ls /content/drive/MyDrive/YelpDataset/models

gpt2-finetuned-yelp-reviews-sample-1cycle-Automotive.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-BeautySpa.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-Food.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-HomeServices.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-Restaurants.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-Shopping.pth


In [ ]:
%cp /content/drive/MyDrive/YelpDataset/models/gpt2-finetuned-yelp-reviews-sample-1cycle-Food.pth /content/

In [ ]:
learn.load("gpt2-finetuned-yelp-reviews-sample-1cycle-Food").cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [ ]:
"""N_INITIAL_WORDS = 5
LEN_SAMPLE_DF = sample_df.shape[0]
N_REVIEWS = 10
BINS = list(range(10,15))"""
N_INITIAL_WORDS = 5
LEN_SAMPLE_DF = sample_df.shape[0]
N_REVIEWS = 2000
BINS = list(range(10,351))

In [ ]:
import warnings
warnings.filterwarnings(action="ignore")
from sklearn.model_selection import train_test_split
generated_df_rows = []
for i in range(len(BINS)-1):
    sample_row_df = sample_df[(sample_df["sentenceLength"]>=BINS[i]) & (sample_df["sentenceLength"]<BINS[i+1])]
    bin_proportion = (sample_row_df.shape[0])/float(LEN_SAMPLE_DF)
    n_reviews_to_generate = int((bin_proportion)*N_REVIEWS)+1
    try:
        or_df, cg_df =  train_test_split(sample_row_df, test_size=0.5, random_state=2021)
    except Exception as e:
        print(e)
        continue
    sample_reviews_df = cg_df.sample(n=n_reviews_to_generate,random_state=2021)
    or_sample_reviews_df = or_df.sample(n=n_reviews_to_generate,random_state=2021)
    if BINS[i] in range(10,21):
        max_len = 20
    else:
        max_len = BINS[i]
    for index, row in sample_reviews_df.iterrows():
        text = row["reviewText"]
        prompt = (" ").join(text.strip().split(" ")[:N_INITIAL_WORDS])
        prompt_ids = tokenizer.encode(prompt)
        inp = tensor(prompt_ids)[None].cuda()
        # set seed to reproduce results
        random_seed(seed_value=2021, use_cuda=True)
        preds = learn.model.generate(inp, max_length=max_len, do_sample=True, top_k=0, top_p=0.92, temperature=0.7)
        decoded_preds = tokenizer.decode(preds[0],skip_special_tokens=True)
        generated_df_rows.append([row["category"],row["overall"],decoded_preds,"CG"])
    for index, row in or_sample_reviews_df.iterrows():
        generated_df_rows.append([row["category"],row["overall"],row["reviewText"],"OR"])
#     break
generated_df = pd.DataFrame(generated_df_rows, columns=["category", "rating", "text", "label"])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

With n_samples=1, test_size=0.5 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

With n_samples=1, test_size=0.5 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

...

With n_samples=1, test_size=0.5 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

With n_samples=1, test_size=0.5 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

With n_samples=1, test_size=0.5 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=1, test_size=0.5 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


In [ ]:
generated_df.shape

(4240, 4)

In [ ]:
generated_df.head()

,category,rating,text,label
0,Food,4.5,Amazingly fresh donuts. You've been warned. The only reason I'm giving 4 stars is,CG
1,Food,5.0,Great beer and service in a very professional manner. I will definitely be back!Very rude.,CG
2,Food,5.0,Wonderful Time!\nStaff Is Excellent!\nWhiskey Is Not an easy task to make,CG
3,Food,4.0,From first hand experience this place has Nashville's best hamburger.,OR
4,Food,4.0,Best and freshest bulk food in Tucson. Best fresh veggies.,OR


In [ ]:
def post_process(str, k=4):
    if not str.endswith("."):
        full_stop_split_str = str.split(".")
        clean_full_stop_split_str = [s.strip() for s in full_stop_split_str if s.strip()]
        if len(clean_full_stop_split_str) > 1:
            last_sentence = clean_full_stop_split_str[-1]
        else:
            return str
        last_sentence_len = len(last_sentence.split(" "))
        if last_sentence_len > k:
            return str
        return (". ").join(clean_full_stop_split_str[:-1]).strip()+(".")
    return str

In [ ]:
generated_df['text_'] = generated_df['text'].apply(lambda x: post_process(x))

In [ ]:
generated_df.head()

,category,rating,text,label,text_
0,Food,4.5,Amazingly fresh donuts. You've been warned. The only reason I'm giving 4 stars is,CG,Amazingly fresh donuts. You've been warned. The only reason I'm giving 4 stars is
1,Food,5.0,Great beer and service in a very professional manner. I will definitely be back!Very rude.,CG,Great beer and service in a very professional manner. I will definitely be back!Very rude.
2,Food,5.0,Wonderful Time!\nStaff Is Excellent!\nWhiskey Is Not an easy task to make,CG,Wonderful Time!\nStaff Is Excellent!\nWhiskey Is Not an easy task to make
3,Food,4.0,From first hand experience this place has Nashville's best hamburger.,OR,From first hand experience this place has Nashville's best hamburger.
4,Food,4.0,Best and freshest bulk food in Tucson. Best fresh veggies.,OR,Best and freshest bulk food in Tucson. Best fresh veggies.


In [ ]:
generated_df.drop(columns=["text"],axis=1,inplace=True)

In [ ]:
generated_df[1500:1512]

,category,rating,label,text_
1500,Food,4.5,CG,"Not impressed; staff wasn't very friendly. The only reason I'm giving 4 stars is the customer service. I had an employee open the door at 6:30 and the store was closed. The cashier had her hand in the door, and it was not ready"
1501,Food,3.5,CG,Had a late night craving for a nice coffee and decided to stop by for a drink. The atmosphere is super cute and the staff was very friendly. There was a lot of seating inside and outside. I got a iced coffee with oat milk and
1502,Food,4.0,CG,World class icecream shop!!! It's an amazing place to try out all the flavors and the staff is always so friendly. \n\nI love that they have a rewards program where you get free icecream and they accept credit cards which is always a plus! This
1503,Food,4.5,CG,"When you charge almost $4 for a single scoop, you're paying a premium for the size you get. \n\nIf you want a small, basic flavor, you can get it at a grocery store in the area. I would suggest getting one if you're"
1504,Food,3.0,CG,"Yum! Had their BLT. Very good. The only reason I didn't give this place a 4 star is because the service was a little slow. Not sure if it was because of the other reviews, but it seemed like a good experience."
1505,Food,3.5,CG,Outstanding service!! I love all the flavors and the taste. I will definitely be back!Very rude. Went in at around 3:15pm and they had a sign on the door that said they were closed. It was clear that they were closed.
1506,Food,4.5,CG,The glazed donut here is the best I've ever had. The staff is friendly and the coffee is top notch. I will definitely be back!I love this place. The cakes are so beautiful and the flavors are so fresh.
1507,Food,3.0,CG,"I used to always love this place, but now they've got a lot of the stuff you can't get at Publix, including the sushi section. \n\nThe store is clean and well stocked. \n\nI do recommend this place to anyone."
1508,Food,4.0,CG,This place makes amazing bubble tea. The fruit tea is good but not spectacular. I would recommend the other two. I think they're the best bubble tea place in town. The customer service is excellent. The staff is extremely friendly.
1509,Food,3.5,CG,"I like this gas station, it's convenient and has a great produce section. I go there to get my oil and vinegar. The prices are reasonable, and the store is clean and well kept. I love the fact that I can get a carwash with my car"


In [ ]:
generated_df.to_csv("Generated_Sentences_Food.csv", index=None)

In [ ]:
%cp /content/Generated_Sentences_Food.csv /content/drive/MyDrive/YelpDataset/generatedData